In [ ]:
%pip install llama-index-llms-huggingface
%pip install llama-index-llms-huggingface-api
%pip install llama-index
%pip install python-dotenv
%pip install llama-index-llms-deepseek
%pip install transformers
%pip install scikit-learn


In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
import re

load_dotenv()  # Load environment variables from .env
from sklearn.metrics import accuracy_score, f1_score
from llama_index.llms.gemini import Gemini

llm = Gemini(model="models/gemini-2.0-flash")
# Test case to see if lllm is working
# response = llm.complete("To infinity, and")
# print(response)

To infinity, and **beyond!**



In [29]:
import pandas as pd

# Load dataset and choose relevant columns
df = pd.read_csv("MH_Campaign_Tweets_Sentiment_Scored.csv")
df = df[["tweet", "tokenised_tweet", "sentiment_scores", "compound_score"]]
df.drop_duplicates(subset=["tweet"], inplace=True)

# Preview the cleaned dataset
print("Cleaned dataset preview:")
print(df.head(5))

Cleaned dataset preview:
                                               tweet  \
0  People with OCD experience intensely negative,...   
1  Hello all I'm Erika an @IOCDF grassroots advoc...   
2  In light of last week's OCD Awareness week, we...   
3  The GIFS now have over 1.4 million views on GI...   
4  The 2022 #OCDweek events, activities, and live...   

                                     tokenised_tweet  \
0  ['people', 'ocd', 'experience', 'intensely', '...   
1  ['hello', 'erika', 'iocdf', 'grassroots', 'adv...   
2  ['light', 'last', 'week', 'ocd', 'awareness', ...   
3  ['gifs', 'million', 'views', 'giphy', 'hope', ...   
4  ['ocdweek', 'events', 'activities', 'livestrea...   

                                    sentiment_scores  compound_score  
0  {'neg': 0.242, 'neu': 0.645, 'pos': 0.113, 'co...         -0.8111  
1  {'neg': 0.043, 'neu': 0.821, 'pos': 0.136, 'co...          0.6981  
2  {'neg': 0.0, 'neu': 0.925, 'pos': 0.075, 'comp...          0.3400  
3  {'neg': 0.0, '

In [ ]:
# Define a function to convert compound sentiment score to Positive/Neutral/Negative label
def score_to_label(compound_score: float) -> str:
    """Convert compound sentiment score to Positive/Neutral/Negative label."""
    if compound_score >= 0.05:
        return "Positive"
    elif compound_score <= -0.05:
        return "Negative"
    else:
        return "Neutral"


# Apply the function to the compound_score column to create a new sentiment_label column
df["sentiment_label"] = df["compound_score"].apply(score_to_label)
print(df.head(5))

                                               tweet  \
0  People with OCD experience intensely negative,...   
1  Hello all I'm Erika an @IOCDF grassroots advoc...   
2  In light of last week's OCD Awareness week, we...   
3  The GIFS now have over 1.4 million views on GI...   
4  The 2022 #OCDweek events, activities, and live...   

                                     tokenised_tweet  \
0  ['people', 'ocd', 'experience', 'intensely', '...   
1  ['hello', 'erika', 'iocdf', 'grassroots', 'adv...   
2  ['light', 'last', 'week', 'ocd', 'awareness', ...   
3  ['gifs', 'million', 'views', 'giphy', 'hope', ...   
4  ['ocdweek', 'events', 'activities', 'livestrea...   

                                    sentiment_scores  compound_score  \
0  {'neg': 0.242, 'neu': 0.645, 'pos': 0.113, 'co...         -0.8111   
1  {'neg': 0.043, 'neu': 0.821, 'pos': 0.136, 'co...          0.6981   
2  {'neg': 0.0, 'neu': 0.925, 'pos': 0.075, 'comp...          0.3400   
3  {'neg': 0.0, 'neu': 0.848, 'pos': 0

In [41]:
aspect_configs = {
    "Staff Friendliness": {
        "keywords": [
            "staff",
            "nurse",
            "doctor",
            "physician",
            "receptionist",
            "hospital staff",
        ],
        "positive_markers": [
            "friendly",
            "kind",
            "helpful",
            "nice",
            "caring",
            "professional",
            "attentive",
        ],
        "negative_markers": [
            "rude",
            "impolite",
            "unprofessional",
            "unfriendly",
            "mean",
            "disrespectful",
        ],
    },
    "Wait Time": {
        "keywords": ["wait", "waiting", "waited", "hours", "minutes", "time"],
        "positive_markers": ["no wait", "quick", "quickly", "short wait", "fast"],
        "negative_markers": ["long wait", "waited for", "hours", "forever", "slow"],
    },
    "Overall Experience": {
        "keywords": ["overall experience", "overall", "experience"],
        "positive_markers": [
            "good",
            "great",
            "excellent",
            "amazing",
            "wonderful",
            "positive",
        ],
        "negative_markers": ["bad", "terrible", "horrible", "awful", "negative"],
    },
    "Medication Feedback": {
        "keywords": [
            "medication",
            "medicine",
            "meds",
            "treatment",
            "drug",
            "prescription",
        ],
        "positive_markers": ["effective", "worked", "helped", "effective", "relief"],
        "negative_markers": [
            "didn't work",
            "not work",
            "ineffective",
            "side effect",
            "worse",
        ],
    },
    "Communication Tone": {
        "keywords": [
            "listen",
            "listened",
            "listening",
            "explained",
            "explain",
            "explanation",
            "communicate",
            "communication",
            "answered",
            "answer",
            "told",
            "said",
        ],
        "positive_markers": [
            "listened",
            "patient",
            "caring",
            "attentive",
            "clear",
            "explained clearly",
            "understood",
        ],
        "negative_markers": [
            "didn't listen",
            "not listen",
            "ignored",
            "rushed",
            "dismissive",
            "confusing",
        ],
    },
}

In [ ]:
from textblob import TextBlob



# Using TextBlob for sentiment analysis as a stand-in for Gemini LLM

def classify_sentiment(tweet: str):
    """

    Analyze a tweet's sentiment using an LLM (simulated here with TextBlob).

    Returns a dict with overall emotion, intensity, and aspect-based sentiment breakdown.
    """

    text = tweet.lower()

    analysis = {}


    # Overall sentiment polarity using TextBlob (as a stand-in for Gemini LLM analysis)

    blob = TextBlob(tweet)

    polarity = (

        blob.sentiment.polarity

    )  # range [-1.0, 1.0], where >0 = positive sentiment, <0 = negative

    # Determine overall emotion category

    if polarity > 0.1:

        emotion = "Positive"

    elif polarity < -0.1:

        emotion = "Negative"

    else:

        emotion = "Neutral"

    analysis["emotion"] = emotion


    # Determine intensity of sentiment

    if emotion == "Positive":

        intensity = "Very Positive" if polarity >= 0.6 else "Positive"

    elif emotion == "Negative":

        intensity = "Very Negative" if polarity <= -0.6 else "Negative"

    else:

        intensity = "Neutral"

    analysis["intensity"] = intensity


    # Aspect-based sentiment analysis

    aspect_sentiments = {}

    for aspect, config in aspect_configs.items():

        # Check if any aspect keyword is present in the tweet text

        if any(keyword in text for keyword in config["keywords"]):

            # If aspect context is found, determine sentiment for that aspect

            aspect_sent = "Neutral"  # default sentiment for aspect

            # Check for explicit negative markers first

            for marker in config["negative_markers"]:

                if marker in text:

                    aspect_sent = "Negative"

                    break

            # If no negative sentiment marker was found, check for positive markers

            if aspect_sent == "Neutral":

                for marker in config["positive_markers"]:

                    if marker in text:

                        aspect_sent = "Positive"

                        break

            aspect_sentiments[aspect] = aspect_sent


    analysis["aspects"] = aspect_sentiments

    return analysis

In [ ]:
# Apply the classifier to each tweet and collect predictions
predicted_emotions = []
for tweet in df["tweet"]:
    result = classify_sentiment(tweet)
    predicted_emotions.append(result["emotion"])

# Use scikit-learn to evaluate the model's predictions against reference labels
y_true = df["sentiment_label"].tolist()  # actual labels from the dataset
y_pred = predicted_emotions  # predicted labels from our classifier

accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="macro")  # macro-averaged F1 for multi-class

print("\nModel Evaluation:")
print("  Accuracy       : {:.2f}%".format(accuracy * 100))
print("  Macro F1-score : {:.2f}%".format(f1 * 100))
print("\nExample Sentiment Analysis on Sample Tweets:\n")


Model Evaluation:
  Accuracy       : 56.07%
  Macro F1-score : 45.51%

Example Sentiment Analysis on Sample Tweets:



In [ ]:
# Prepare to display output with emojis. If no aspect is relevant, we'll show only overall sentiment.
# Define mapping from sentiment label to emoji for convenience
emotion_emoji = {"Positive": "😊", "Negative": "😡", "Neutral": "🤔"}

# Show a few example tweet outputs with sentiment breakdown and emojis
# We'll pick one example from each sentiment category (if available) to demonstrate.
examples = []
for sentiment in ["Positive", "Negative", "Neutral"]:
    # find index of a tweet where the *predicted* emotion matches this sentiment
    try:
        idx = y_pred.index(sentiment)
        examples.append(idx)
    except ValueError:
        continue  # sentiment not present in predictions

# Remove duplicates (in case the first found index overlaps, e.g., if first tweet is positive and index 0 used twice)
examples = sorted(set(examples))
print("Example sentiment analysis on sample tweets:\n")
for idx in examples:
    tweet_text = df.loc[idx, "tweet"]
    result = classify_sentiment(tweet_text)
    overall = result["emotion"]
    intensity = result["intensity"]
    aspects = result["aspects"]

    # Print tweet text with a header
    print("Tweet:")
    print(tweet_text)
    print()  # Blank line for spacing

    # Print overall sentiment and intensity with emojis
    print("Overall Sentiment: {} {}".format(overall, emotion_emoji.get(overall, "")))
    print("Intensity        : {} {}".format(intensity, emotion_emoji.get(overall, "")))

    # Print aspect sentiments (if any)
    if aspects:
        print("Aspect Sentiments:")
        for aspect, sentiment in aspects.items():
            emoji = (
                "✅"
                if sentiment == "Positive"
                else "❌" if sentiment == "Negative" else "🤔"
            )
            print("  - {}: {} {}".format(aspect, sentiment, emoji))
    else:
        print("Aspect Sentiments: (No specific aspects mentioned)")

    # Print a separator line
    print("-" * 50)

Example sentiment analysis on sample tweets:

Tweet:
People with OCD experience intensely negative, repetitive and intrusive thoughts with a chronic feeling of doubt or danger. To calm the anxiety, people with OCD will often repeat an action, these are known as compulsions. If you experience this, therapy can help. #OCDWeek #OCD https://t.co/2MKew1wG4u

Overall Sentiment: Neutral 🤔
Intensity        : Neutral 🤔
Aspect Sentiments:
  - Overall Experience: Negative ❌
--------------------------------------------------
Tweet:
Hello all I'm Erika an @IOCDF grassroots advocate that loves to #rockyourvalues by painting rocks to spread awareness and lower stigma about OCD in our community.  Here is a look back from #OCDweek 2022!! Excited for my workshop at the end of Feb! 🫶

https://t.co/nzsi43m1gW

Overall Sentiment: Positive 😊
Intensity        : Positive 😊
Aspect Sentiments: (No specific aspects mentioned)
--------------------------------------------------
Tweet:
No one should have to convinc

In [ ]:
# # Testing gemma 3
# from IPython.display import display, Markdown, clear_output
# import ollama


# # Example prompt


# prompt = "What is pandas dataframe?"


# # Call llama.chat with streaming enabled


# # Replace 'c' with your LLM client instance


# messages = [


#     {"role": "user", "content": prompt},


# ]


# response_stream = ollama.chat(


#     model="gemma3:latest", messages=[{"role": "user", "content": prompt}], stream=True


# )


# # Initialize an empty response string


# streamed_response = ""


# # Iterate over the streamed tokens


# for token in response_stream:


#     # Append the current token to the response


#     streamed_response += token["message"]["content"]


#     # Clear previous output and display the updated response


#     clear_output(wait=True)


#     display(Markdown("**LLM Response (Streaming)**\n\n" + streamed_response))

**LLM Response (Streaming)**

Okay, let's break down what a Pandas DataFrame is. It's a fundamental data structure in Python, and it's incredibly powerful for working with structured data.

**What is a Pandas DataFrame?**

At its core, a Pandas DataFrame is like a spreadsheet or a SQL table in Python.  It's a two-dimensional, labeled data structure with columns of potentially different data types.  Think of it as a table where:

* **Rows:** Represent individual records or observations.
* **Columns:** Represent different attributes or features for those records.
* **Labels:** Each row and column has a label (name) that you can use to access and manipulate the data.

**Key Characteristics and Features:**

1. **Data Types:**  Each column in a DataFrame can hold a different data type (e.g., integers, floats, strings, booleans, dates). This flexibility is a major strength.

2. **Labeled Axes:**  The DataFrame has two primary labels:
   * **Index:**  Labels for the rows.  This is often (but not always) a numerical sequence, but can be customized.
   * **Columns:** Labels for the columns.

3. **Heterogeneous Columns:**  As mentioned, columns don't have to be the same type.  A DataFrame might have columns for 'age' (integers), 'name' (strings), and 'salary' (floats).

4. **Data Alignment:** Pandas automatically aligns data based on the column labels, making it easy to perform operations on columns with similar names.

5. **Powerful Operations:** Pandas provides a vast array of functions and methods for:
   * **Data Cleaning:** Handling missing values, removing duplicates, etc.
   * **Data Transformation:**  Filtering, sorting, grouping, applying functions to columns.
   * **Data Analysis:**  Calculating statistics, creating visualizations.
   * **Data I/O:** Reading data from various file formats (CSV, Excel, SQL databases, etc.) and writing data to those formats.

**How to Create a DataFrame:**

There are several ways to create a DataFrame:

* **From a Dictionary:**
   ```python
   import pandas as pd

   data = {'Name': ['Alice', 'Bob', 'Charlie'],
           'Age': [25, 30, 28],
           'City': ['New York', 'London', 'Paris']}

   df = pd.DataFrame(data)
   print(df)
   ```

* **From a List of Lists:**
   ```python
   import pandas as pd

   data = [['Alice', 25, 'New York'],
           ['Bob', 30, 'London'],
           ['Charlie', 28, 'Paris']]

   df = pd.DataFrame(data, columns=['Name', 'Age', 'City']) # Specify column names
   print(df)
   ```

* **From a NumPy Array:**
   ```python
   import pandas as pd
   import numpy as np

   data = np.random.randn(5, 3)  # Create a 5x3 array of random numbers
   df = pd.DataFrame(data, columns=['Col1', 'Col2', 'Col3'])
   print(df)
   ```

**Example Output (from the dictionary example):**

```
      Name  Age      City
0    Alice   25  New York
1      Bob   30    London
2  Charlie   28     Paris
```

**Why is Pandas DataFrames so popular?**

* **Ease of Use:** Pandas provides a high-level, intuitive interface for working with data.
* **Performance:**  It's built on NumPy, which is optimized for numerical computations.
* **Integration:** It seamlessly integrates with other Python libraries like NumPy, SciPy, Matplotlib, and Scikit-learn.

**Resources for Learning More:**

* **Pandas Documentation:** [https://pandas.pydata.org/docs/](https://pandas.pydata.org/docs/)
* **Tutorials:** [https://www.datacamp.com/tutorial/pandas](https://www.datacamp.com/tutorial/pandas)
* **Real Python Pandas Tutorial:** [https://realpython.com/pandas-dataframe-quickstart/](https://realpython.com/pandas-dataframe-quickstart/)


Do you want me to:

*   Give you more specific examples of operations you can perform with DataFrames?
*   Explain how DataFrames relate to NumPy?
*   Help you with a particular task you're trying to accomplish with Pandas?